In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import save_npz, load_npz
import numpy as np
import pickle
from google.colab import drive

# Загрузите ваш датасет
books = pd.read_csv('clean_books.csv')

# ## Шаг 3: Создание и сохранение TF-IDF модели
# %%
# Инициализация TF-IDF
tfidf = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1, 2),
    max_features=10000  # Можно увеличить для полного датасета
)

# Создание матрицы
print("Создание TF-IDF матрицы...")
tfidf_matrix = tfidf.fit_transform(books['title'].fillna(''))

# Сохранение модели и матрицы
print("Сохранение артефактов...")
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)
save_npz('tfidf_matrix.npz', tfidf_matrix)

# ## Шаг 4: Вычисление и сохранение матрицы схожести (по частям)
# %%
# Размер чанка (подбирается в зависимости от доступной RAM)
chunk_size = 2000
n_books = len(books)
similarity_matrix = np.zeros((n_books, n_books), dtype=np.float32)

for i in range(0, n_books, chunk_size):
    for j in range(0, n_books, chunk_size):
        print(f"Обработка блока {i//chunk_size+1}x{j//chunk_size+1}...")
        chunk_sim = cosine_similarity(
            tfidf_matrix[i:i+chunk_size],
            tfidf_matrix[j:j+chunk_size]
        )
        similarity_matrix[i:i+chunk_size, j:j+chunk_size] = chunk_sim

# Сохранение матрицы схожести
np.save('similarity_matrix.npy', similarity_matrix)

# ## Шаг 5: Сохранение метаданных
# %%
# Сохраняем индексы для быстрого поиска
book_indices = pd.Series(books.index, index=books['title'].str.lower()).to_dict()
with open('book_indices.pkl', 'wb') as f:
    pickle.dump(book_indices, f)

print("Все компоненты успешно сохранены!")

Выходные данные были обрезаны до нескольких последних строк (5000).
Обработка блока 100x33...
Обработка блока 100x34...
Обработка блока 100x35...
Обработка блока 100x36...
Обработка блока 100x37...
Обработка блока 100x38...
Обработка блока 100x39...
Обработка блока 100x40...
Обработка блока 100x41...
Обработка блока 100x42...
Обработка блока 100x43...
Обработка блока 100x44...
Обработка блока 100x45...
Обработка блока 100x46...
Обработка блока 100x47...
Обработка блока 100x48...
Обработка блока 100x49...
Обработка блока 100x50...
Обработка блока 100x51...
Обработка блока 100x52...
Обработка блока 100x53...
Обработка блока 100x54...
Обработка блока 100x55...
Обработка блока 100x56...
Обработка блока 100x57...
Обработка блока 100x58...
Обработка блока 100x59...
Обработка блока 100x60...
Обработка блока 100x61...
Обработка блока 100x62...
Обработка блока 100x63...
Обработка блока 100x64...
Обработка блока 100x65...
Обработка блока 100x66...
Обработка блока 100x67...
Обработка блока 100x68

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import save_npz
import numpy as np
import pickle
from google.colab import files
from tqdm import tqdm
import time
from sklearn.preprocessing import normalize

# Инициализация таймера
start_global = time.time()

# Загружаем датасет с прогресс-баром
print("1. Загрузка датасета...")
start = time.time()
books = pd.read_csv('clean_books.csv')
print(f"Загружено {len(books)} книг за {time.time() - start:.2f} секунд")

# Инициализация TF-IDF
print("\n2. Инициализация TF-IDF векторизатора...")
tfidf = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000,
    dtype=np.float32
)

# Создание TF-IDF матрицы с прогресс-баром
print("\n3. Создание TF-IDF матрицы...")
start = time.time()
tfidf_matrix = tfidf.fit_transform(tqdm(books['title'].fillna(''), desc="Обработка текстов"))
print(f"TF-IDF матрица создана за {time.time() - start:.2f} секунд")
print(f"Размер матрицы: {tfidf_matrix.shape} (книги x фичи)")

# Сохранение модели TF-IDF
print("\n4. Сохранение артефактов TF-IDF...")
start = time.time()
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)
save_npz('tfidf_matrix.npz', tfidf_matrix)
print(f"Артефакты сохранены за {time.time() - start:.2f} секунд")

# Параметры для топ-рекомендаций
top_k = 5
n_books = len(books)
chunk_size = 20000

# Матрицы для хранения рекомендаций
top_indices = np.zeros((n_books, top_k), dtype=np.int32)
top_scores = np.zeros((n_books, top_k), dtype=np.float32)

# Вычисляем рекомендации по чанкам с прогресс-баром
print(f"\n5. Вычисление топ-{top_k} рекомендаций для {n_books} книг...")
total_chunks = (n_books + chunk_size - 1) // chunk_size

for i in tqdm(range(0, n_books, chunk_size), desc="Обработка чанков", total=total_chunks):
    chunk_start = i
    chunk_end = min(i + chunk_size, n_books)

    # Вычисляем схожесть для текущего чанка
    chunk_sim = cosine_similarity(tfidf_matrix[chunk_start:chunk_end], tfidf_matrix)

    # Обрабатываем каждую книгу в чанке
    for k in tqdm(range(chunk_sim.shape[0]), desc=f"Книги {chunk_start}-{chunk_end}", leave=False):
        book_idx = i + k
        sim_scores = chunk_sim[k]
        sim_scores[book_idx] = -1  # Исключаем текущую книгу

        # Находим топ-рекомендации
        top_5_indices = np.argpartition(sim_scores, -top_k)[-top_k:]
        top_5_scores = sim_scores[top_5_indices]

        # Сортируем
        sorted_indices = np.argsort(-top_5_scores)
        top_indices[book_idx] = top_5_indices[sorted_indices]
        top_scores[book_idx] = top_5_scores[sorted_indices]

# Сохранение результатов
print("\n6. Сохранение результатов...")
start = time.time()
np.save('top_5_indices.npy', top_indices)
np.save('top_5_scores.npy', top_scores)
print(f"Результаты сохранены за {time.time() - start:.2f} секунд")

# Сохранение метаданных
print("\n7. Сохранение метаданных...")
start = time.time()
book_indices = pd.Series(books.index, index=books['title'].str.lower()).to_dict()
with open('book_indices.pkl', 'wb') as f:
    pickle.dump(book_indices, f)
print(f"Метаданные сохранены за {time.time() - start:.2f} секунд")

# Итоговая информация
total_time = time.time() - start_global
print("\n" + "="*50)
print("Все компоненты успешно сохранены!")
print(f"Общее время выполнения: {total_time/60:.2f} минут")
print(f"Размер полной матрицы схожести: {n_books}x{n_books} = {n_books**2:,} элементов")
print(f"Размер сохраненных данных: {n_books}x{top_k} x2 = {n_books*top_k*2:,} элементов")
print("="*50)

# Опционально: скачиваем файлы
# print("\n8. Скачивание файлов...")
# files.download('top_5_indices.npy')
# files.download('top_5_scores.npy')
# files.download('book_indices.pkl')
# files.download('tfidf_vectorizer.pkl')
# files.download('tfidf_matrix.npz')

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import save_npz, csr_matrix
import numpy as np
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
import time

# Оптимизированная функция обработки книги
def process_book(args):
    """Обработка одной книги с распаковкой аргументов"""
    k, chunk_sim, i, top_k = args
    book_idx = i + k
    sim_scores = chunk_sim[k].copy()
    sim_scores[book_idx] = -1
    top_indices = np.argpartition(sim_scores, -top_k)[-top_k:]
    top_scores = sim_scores[top_indices]
    sorted_indices = np.argsort(-top_scores)
    return book_idx, top_indices[sorted_indices], top_scores[sorted_indices]

# Основной код
start_global = time.time()

# 1. Загрузка данных
print("1. Загрузка датасета...")
books = pd.read_csv('clean_books.csv')

# 2. TF-IDF с оптимизированными параметрами
print("\n2. Создание TF-IDF матрицы...")
tfidf = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1, 1),
    max_features=5000,
    dtype=np.float32
)
tfidf_matrix = tfidf.fit_transform(books['title'].fillna(''))
tfidf_matrix = csr_matrix(tfidf_matrix, dtype=np.float32)

# 3. Сохранение моделей
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)
save_npz('tfidf_matrix.npz', tfidf_matrix)

# 4. Параметры
top_k = 5
n_books = len(books)
chunk_size = 20000
top_indices = np.zeros((n_books, top_k), dtype=np.int32)
top_scores = np.zeros((n_books, top_k), dtype=np.float32)

# 5. Основной цикл с параллельной обработкой
print(f"\n3. Вычисление рекомендаций для {n_books} книг...")
for i in tqdm(range(0, n_books, chunk_size), desc="Чанки"):
    chunk_end = min(i + chunk_size, n_books)
    chunk_sim = linear_kernel(tfidf_matrix[i:chunk_end], tfidf_matrix)

    # Подготовка аргументов для параллельной обработки
    args_list = [(k, chunk_sim, i, top_k) for k in range(chunk_sim.shape[0])]

    # Параллельная обработка с обработкой ошибок
    try:
        results = Parallel(n_jobs=-1, verbose=0)(
            delayed(process_book)(args) for args in args_list
        )

        for book_idx, indices, scores in results:
            top_indices[book_idx] = indices
            top_scores[book_idx] = scores

    except Exception as e:
        print(f"\nОшибка при обработке чанка {i}-{chunk_end}: {str(e)}")
        # Альтернативная обработка без параллелизма при ошибке
        for k in range(chunk_sim.shape[0]):
            book_idx = i + k
            sim_scores = chunk_sim[k].copy()
            sim_scores[book_idx] = -1
            top_5_indices = np.argpartition(sim_scores, -top_k)[-top_k:]
            top_5_scores = sim_scores[top_5_indices]
            sorted_indices = np.argsort(-top_5_scores)
            top_indices[book_idx] = top_5_indices[sorted_indices]
            top_scores[book_idx] = top_5_scores[sorted_indices]

# 6. Сохранение результатов
print("\n4. Сохранение результатов...")
np.save('top_5_indices.npy', top_indices)
np.save('top_5_scores.npy', top_scores)

# Сохранение метаданных
book_indices = pd.Series(books.index, index=books['title'].str.lower()).to_dict()
with open('book_indices.pkl', 'wb') as f:
    pickle.dump(book_indices, f)

# Вывод времени выполнения
total_time = (time.time()-start_global)/60
print(f"\nГотово! Общее время: {total_time:.1f} минут")
print(f"Обработано {n_books} книг")
print(f"Средняя скорость: {n_books/total_time/60:.1f} книг/сек")

1. Загрузка датасета...

2. Создание TF-IDF матрицы...

3. Вычисление рекомендаций для 271379 книг...


Чанки: 100%|██████████| 14/14 [19:39<00:00, 84.23s/it]



4. Сохранение результатов...

Готово! Общее время: 19.7 минут
Обработано 271379 книг
Средняя скорость: 229.4 книг/сек


In [5]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class ContentRecommender:
    def __init__(self,
                 books_path='clean_books.csv',
                 tfidf_vectorizer_path='tfidf_vectorizer.pkl',
                 tfidf_matrix_path='tfidf_matrix.npz',
                 top_indices_path='top_5_indices.npy',
                 top_scores_path='top_5_scores.npy',
                 book_indices_path='book_indices.pkl'):

        # Загрузка данных с обработкой дубликатов
        self.books = pd.read_csv(books_path).drop_duplicates(subset=['title'])

        # Загрузка моделей с обработкой версий
        try:
            with open(tfidf_vectorizer_path, 'rb') as f:
                self.vectorizer = pickle.load(f)
        except Exception as e:
            print(f"Ошибка загрузки векторайзера: {e}. Инициализируем новый.")
            self.vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))

        self.tfidf_matrix = load_npz(tfidf_matrix_path)
        self.top_indices = np.load(top_indices_path)
        self.top_scores = np.load(top_scores_path)

        with open(book_indices_path, 'rb') as f:
            self.book_indices = pickle.load(f)

    def get_recommendations(self, book_title, top_n=5):
        book_title_lower = book_title.lower()

        if book_title_lower not in self.book_indices:
            return f"Книга '{book_title}' не найдена."

        idx = self.book_indices[book_title_lower]
        recommended_indices = self.top_indices[idx][:top_n]
        recommended_scores = self.top_scores[idx][:top_n]

        recommendations = self.books.iloc[recommended_indices][['title', 'author', 'year']]
        recommendations['similarity'] = recommended_scores

        # Удаление дубликатов и самой книги из рекомендаций
        recommendations = recommendations[
            (recommendations['title'].str.lower() != book_title_lower)
        ].drop_duplicates(subset=['title'])

        return recommendations.reset_index(drop=True)

if __name__ == '__main__':
    recommender = ContentRecommender()
    test_books = [
        "Harry Potter and the Sorcerer's Stone",
        "The Great Gatsby",
        "Nonexistent Book"
    ]

    for book in test_books:
        print(f"\nРекомендации для '{book}':")
        recs = recommender.get_recommendations(book)
        print(recs if isinstance(recs, pd.DataFrame) else recs)


Рекомендации для 'Harry Potter and the Sorcerer's Stone':
                                               title                author  \
0                                   Scheme of Things     R., Barri Flowers   
1                     Spellbound (Harlequin Romance)          Margaret Way   
2  The Okinawa Program: How the World's Longest-L...    Bradley J. Willcox   
3  Essential Calculus: With Applications (Dover B...  Richard A. Silverman   
4                                   One Lonely Night         Susan Kay Law   

   year  similarity  
0  2002    1.000000  
1  1983    1.000000  
2  2001    0.969923  
3  1989    0.969923  
4  1997    0.969923  

Рекомендации для 'The Great Gatsby':
                                               title  \
0                              The Work of Her Hands   
1  Clay's Quilt: A Novel (Ballantine Reader's Cir...   
2  Fearless Jones (Fearless Jones Novels (Paperba...   
3                Never Alone (Love Inspired , No 30)   
4  Field Guide to the 

In [6]:

import os
import numpy as np
import pandas as pd
import pickle
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.metrics import pairwise_distances
from scipy.sparse import save_npz
import cupy as cp
import cudf

# Проверка доступности GPU
print("Доступно GPU:", cp.cuda.runtime.getDeviceCount())

books = pd.read_csv('clean_books.csv')
books_cudf = cudf.DataFrame.from_pandas(books)

books_pd = books_cudf.to_pandas()

# Инициализация TF-IDF на GPU
print("Инициализация TF-IDF...")
tfidf = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1, 2),
    max_features=10000
)

# Создание TF-IDF матрицы на GPU
print("Создание TF-IDF матрицы...")
tfidf_matrix = tfidf.fit_transform(books_cudf['title'].fillna(''))

# Конвертация в разреженную матрицу CuPy для эффективных вычислений
tfidf_matrix_gpu = cp.sparse.csr_matrix(tfidf_matrix)

# Параметры рекомендаций
top_k = 5
n_books = len(books_cudf)
chunk_size = 2000  # Размер чанка для обработки

# Выделение памяти на GPU для результатов
top_indices = cp.zeros((n_books, top_k), dtype=cp.int32)
top_scores = cp.zeros((n_books, top_k), dtype=cp.float32)

# Функция для обработки чанка
def process_chunk(start_idx, end_idx):
    chunk_sim = 1 - pairwise_distances(
        tfidf_matrix_gpu[start_idx:end_idx],
        tfidf_matrix_gpu,
        metric='cosine'
    )

    for k in range(chunk_sim.shape[0]):
        book_idx = start_idx + k
        sim_scores = chunk_sim[k]
        sim_scores[book_idx] = -1  # Исключаем текущую книгу

        # Находим топ-5 на GPU
        top_5_indices = cp.argpartition(sim_scores, -top_k)[-top_k:]
        top_5_scores = sim_scores[top_5_indices]

        # Сортируем
        sorted_indices = cp.argsort(-top_5_scores)
        top_indices[book_idx] = top_5_indices[sorted_indices]
        top_scores[book_idx] = top_5_scores[sorted_indices]

# Вычисление рекомендаций по чанкам
print("Вычисление рекомендаций...")
for i in range(0, n_books, chunk_size):
    end_idx = min(i + chunk_size, n_books)
    print(f"Обработка книг {i}-{end_idx}...")
    process_chunk(i, end_idx)

# Перенос результатов на CPU
print("Перенос результатов на CPU...")
top_indices_cpu = cp.asnumpy(top_indices)
top_scores_cpu = cp.asnumpy(top_scores)

# Сохранение результатов
print("Сохранение результатов...")
np.save('top_5_indices.npy', top_indices_cpu)
np.save('top_5_scores.npy', top_scores_cpu)

# Сохранение модели TF-IDF
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# Сохранение TF-IDF матрицы
save_npz('tfidf_matrix.npz', tfidf_matrix)

# Сохранение метаданных
book_indices = pd.Series(books_pd.index, index=books_pd['title'].str.lower()).to_dict()
with open('book_indices.pkl', 'wb') as f:
    pickle.dump(book_indices, f)

print("Готово! Все файлы сохранены.")

# Опционально: скачивание файлов (для Google Colab)
# files.download('top_5_indices.npy')
# files.download('top_5_scores.npy')
# files.download('tfidf_vectorizer.pkl')
# files.download('tfidf_matrix.npz')
# files.download('book_indices.pkl')

ModuleNotFoundError: No module named 'cuml'

In [1]:
!pip install --pre scikit-surprise -f https://surpriselib.com/wheels/

Looking in links: https://surpriselib.com/wheels/


In [5]:
pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469552 sha256=336a0a0bcbc17be42d53f8685440a3ff3111a5c424448d7f1bb3bfd8b2f4f74b
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [3]:
# recommenderSVD.py

import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import pickle

class SVDRecommender:
    def __init__(self, ratings_path='clean_ratings.csv', books_path='clean_books.csv'):
        self.ratings_df = pd.read_csv(ratings_path)
        self.books_df = pd.read_csv(books_path)
        self.model = None
        self.trainset = None
        self.reader = Reader(rating_scale=(1, 10))

    def train(self):
        data = Dataset.load_from_df(self.ratings_df[['user_id', 'isbn', 'rating']], self.reader)
        trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
        self.model = SVD(n_factors=100, random_state=42)
        self.model.fit(trainset)
        predictions = self.model.test(testset)
        rmse = accuracy.rmse(predictions)
        print(f'✅ Model trained. RMSE: {rmse:.4f}')
        self.trainset = trainset

        # Сохраняем модель
        with open('svd_model.pkl', 'wb') as f:
            pickle.dump(self.model, f)

    def load_model(self):
        with open('svd_model.pkl', 'rb') as f:
            self.model = pickle.load(f)

    def recommend_books(self, user_id, top_n=10):
        if self.model is None or self.trainset is None:
            raise Exception("Сначала обучите или загрузите модель.")

        all_books = self.books_df['isbn'].unique()
        rated_books = self.ratings_df[self.ratings_df['user_id'] == user_id]['isbn'].tolist()
        books_to_predict = [isbn for isbn in all_books if isbn not in rated_books]

        predictions = [
            (isbn, self.model.predict(user_id, isbn).est)
            for isbn in books_to_predict
        ]
        predictions.sort(key=lambda x: x[1], reverse=True)
        top_books = predictions[:top_n]

        result_df = self.books_df[self.books_df['isbn'].isin([isbn for isbn, _ in top_books])]
        result_df = result_df[['title', 'author', 'year']]
        return result_df.reset_index(drop=True)


# Пример запуска
if __name__ == '__main__':
    recommender = SVDRecommender()
    recommender.train()
    recommendations = recommender.recommend_books(user_id=276729, top_n=5)
    print("Рекомендации:")
    print(recommendations)


RMSE: 1.6399
✅ Model trained. RMSE: 1.6399
Рекомендации:
                                             title            author  year
0                      Dune (Remembering Tomorrow)     Frank Herbert  1996
1   The Two Towers (The Lord of the Rings, Part 2)  J. R. R. Tolkien  1999
2                     Weirdos From Another Planet!    Bill Watterson  1990
3  52 Deck Series: 52 Ways to Celebrate Friendship       Lynn Gordon  2000
4     My Sister's Keeper : A Novel (Picoult, Jodi)      Jodi Picoult  2004


In [ ]:
import pandas as pd
import numpy as np
import pickle
from scipy.sparse import load_npz, hstack
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from tqdm import tqdm
tqdm.pandas()
def load_models():
    """Загрузка предобученных моделей"""
    models = {}

    # Загрузка TF-IDF модели
    with open('tfidf_vectorizer.pkl', 'rb') as f:
        models['tfidf'] = pickle.load(f)
    models['tfidf_matrix'] = load_npz('tfidf_matrix.npz')

    # Загрузка SVD модели
    with open('svd_model.pkl', 'rb') as f:
        models['svd'] = pickle.load(f)

    # Загрузка дополнительных данных
    models['top_indices'] = np.load('top_5_indices.npy')
    models['top_scores'] = np.load('top_5_scores.npy')
    with open('book_indices.pkl', 'rb') as f:
        models['book_indices'] = pickle.load(f)

    return models

def generate_hybrid_features(books_path='clean_books.csv'):
    """Генерация гибридных признаков"""
    # Загрузка данных
    books = pd.read_csv(books_path)

    # Заполнение пропущенных значений
    books['title'] = books['title'].fillna('')
    books['publisher'] = books['publisher'].fillna('Unknown')
    books['author'] = books['author'].fillna('Unknown')
    books['year'] = books['year'].fillna(0).astype(int)

    # Загрузка предобученных моделей
    models = load_models()

    # 1. Используем готовую TF-IDF матрицу
    tfidf_features = models['tfidf_matrix']

    # 2. One-hot кодирование для авторов и издателей
    books['author_simplified'] = books['author'].apply(
        lambda x: x if books['author'].value_counts()[x] > 10 else 'Other')
    books['publisher_simplified'] = books['publisher'].apply(
        lambda x: x if books['publisher'].value_counts()[x] > 10 else 'Other')

    ohe = OneHotEncoder(handle_unknown='ignore', sparse=True)
    categorical_features = ohe.fit_transform(books[['author_simplified', 'publisher_simplified']])

    # 3. Масштабирование года издания
    scaler = MinMaxScaler()
    year_scaled = scaler.fit_transform(books[['year']])

    # 4. Добавляем SVD факторы (если нужно)
    # Для примера - используем первые 50 скрытых факторов
    svd_features = models['svd'].qi[:, :50]  # Факторы для книг

    # Объединение всех признаков
    X_hybrid = hstack([
        tfidf_features,
        categorical_features,
        year_scaled,
        svd_features
    ])

    # Сохранение результатов
    with open('X_hybrid_enhanced.pkl', 'wb') as f:
        pickle.dump({
            'X_hybrid': X_hybrid,
            'isbn_list': books['isbn'].tolist(),
            'title_list': books['title'].tolist(),
            'books_df': books,
            'onehot_encoder': ohe,
            'scaler': scaler,
            'feature_types': ['tfidf', 'categorical', 'year', 'svd_factors']
        }, f)

    print("✅ Гибридные признаки сохранены в X_hybrid_enhanced.pkl")
    print(f"Общая размерность признаков: {X_hybrid.shape[1]}")
    return X_hybrid

if __name__ == '__main__':
    generate_hybrid_features()

In [12]:
import pandas as pd
import numpy as np
import pickle
from scipy.sparse import load_npz, hstack
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from tqdm import tqdm
tqdm.pandas()
def load_models():
    """Загрузка предобученных моделей"""
    models = {}

    # Загрузка TF-IDF модели
    with open('tfidf_vectorizer.pkl', 'rb') as f:
        models['tfidf'] = pickle.load(f)
    models['tfidf_matrix'] = load_npz('tfidf_matrix.npz')

    # Загрузка SVD модели
    with open('svd_model.pkl', 'rb') as f:
        models['svd'] = pickle.load(f)

    # Загрузка дополнительных данных
    models['top_indices'] = np.load('top_5_indices.npy')
    models['top_scores'] = np.load('top_5_scores.npy')
    with open('book_indices.pkl', 'rb') as f:
        models['book_indices'] = pickle.load(f)

    return models

def generate_hybrid_features(books_path='clean_books.csv'):
    """Генерация гибридных признаков"""
    # Загрузка данных
    books = pd.read_csv(books_path)

    # Заполнение пропущенных значений
    books['title'] = books['title'].fillna('')
    books['publisher'] = books['publisher'].fillna('Unknown')
    books['author'] = books['author'].fillna('Unknown')
    books['year'] = books['year'].fillna(0).astype(int)

    # Загрузка предобученных моделей
    models = load_models()

    # 1. Используем готовую TF-IDF матрицу
    tfidf_features = models['tfidf_matrix']

    # 2. One-hot кодирование для авторов и издателей
    author_counts = books['author'].value_counts()
    publisher_counts = books['publisher'].value_counts()

    # Векторизованная обработка (в 100+ раз быстрее)
    books['author_simplified'] = np.where(
        books['author'].map(author_counts) > 10,
        books['author'],
        'Other'
    )

    books['publisher_simplified'] = np.where(
        books['publisher'].map(publisher_counts) > 10,
        books['publisher'],
        'Other'
    )

    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
    categorical_features = ohe.fit_transform(books[['author_simplified', 'publisher_simplified']])

    # 3. Масштабирование года издания
    scaler = MinMaxScaler()
    year_scaled = scaler.fit_transform(books[['year']])

    # 4. Добавляем SVD факторы (если нужно)
    # Для примера - используем первые 50 скрытых факторов
    n_books = tfidf_features.shape[0]  # Получаем общее количество книг из TF-IDF
    svd_n_books = models['svd'].qi.shape[0]  # Количество книг в SVD модели
    if svd_n_books < n_books:
        # Дополняем нулями недостающие строки
        padding = np.zeros((n_books - svd_n_books, 50))
        svd_features = np.vstack([models['svd'].qi[:, :50], padding])
    else:
        svd_features = models['svd'].qi[:n_books, :50]

    # Объединение всех признаков
    X_hybrid = hstack([
        tfidf_features,
        categorical_features,
        year_scaled,
        svd_features
    ])

    # Сохранение результатов
    with open('X_hybrid_enhanced.pkl', 'wb') as f:
        pickle.dump({
            'X_hybrid': X_hybrid,
            'isbn_list': books['isbn'].tolist(),
            'title_list': books['title'].tolist(),
            'books_df': books,
            'onehot_encoder': ohe,
            'scaler': scaler,
            'feature_types': ['tfidf', 'categorical', 'year', 'svd_factors']
        }, f)

    print("✅ Гибридные признаки сохранены в X_hybrid_enhanced.pkl")
    print(f"Общая размерность признаков: {X_hybrid.shape[1]}")
    return X_hybrid

if __name__ == '__main__':
    generate_hybrid_features()

✅ Гибридные признаки сохранены в X_hybrid_enhanced.pkl
Общая размерность признаков: 10938


In [5]:
# Переобучите SVD модель на том же наборе книг, что и TF-IDF

print(f"Предупреждение: SVD обучена на {svd_n_books} книгах, а TF-IDF на {n_books}")
print("Рекомендуется переобучить SVD на полном наборе данных")

NameError: name 'svd_n_books' is not defined

In [7]:
import pickle
with open('X_hybrid_enhanced.pkl', 'rb') as f:
    data = pickle.load(f)
    print(data.keys())

dict_keys(['X_hybrid', 'isbn_list', 'title_list', 'books_df', 'onehot_encoder', 'scaler', 'feature_types'])


In [13]:
import pickle
from surprise import SVD

# Загружаем модель как обычно
with open('svd_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Пересохраняем веса модели в универсальном формате
model_weights = {
    'pu': model.pu,  # user factors
    'qi': model.qi,  # item factors
    'bu': model.bu,  # user biases
    'bi': model.bi,  # item biases
}

# Сохраняем веса в JSON или numpy-формате
import numpy as np
np.savez('svd_weights.npz', **model_weights)